In [7]:
# To pkeras_model=None training, we import the necessary functions and submodules from keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dropout, Reshape, Dense, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adadelta, SGD, RMSprop;
import keras.losses;
from keras.constraints import maxnorm;
from keras.utils import normalize, to_categorical
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, precision_recall_curve, auc

# plt.use('Agg')
from sklearn.model_selection import GridSearchCV, train_test_split



Using TensorFlow backend.


In [39]:
def load_data():
    df = pd.read_csv("eif3a_full_m6aReader2.csv")
    # print(df)
    n = len(df.columns)
    train = int(n / 2)
    x_train = df.iloc[:, 2:train]

    x_val = df.iloc[:, (train + 1):(n - 1)]
    x_val = pd.DataFrame(x_val)
    x_val = x_val.dropna()
    # print(x_val)

    # x_train = np.expand_dims(x_train, axis=1)
    # x_val = np.expand_dims(x_val, axis=1)

    y_train = df.iloc[:, train:train + 1]
    y_val = df.iloc[:, (n - 1):]
    y_val = DataFrame(y_val)
    y_val = y_val.dropna()
    y_val = DataFrame(y_val, dtype=int)

    x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5)

    # print(x_val.shape)
    print(x_train)
    # print(x_test.shape)
    # print(y_val.shape)
    print(y_train)
    # print(y_test.shape)

    return x_train, x_test, x_val, y_test, y_train, y_val

In [41]:
x_train, x_test, x_val, y_test, y_train, y_val = load_data()
x_train.shape[1::]

(151, 105)
      UTR5.x  Start_codons.x  TSS.x  TSS_A.x  exon_stop.x  alternative_exon.x  \
0       True           False  False    False         True               False   
1      False           False  False    False         True               False   
2       True           False  False    False         True               False   
3       True           False  False    False         True               False   
4       True           False  False    False         True               False   
...      ...             ...    ...      ...          ...                 ...   
1205   False           False  False    False        False               False   
1206   False           False  False    False        False               False   
1207   False           False  False    False        False               False   
1208   False           False  False    False        False               False   
1209   False           False  False    False        False               False   

      constituti

(105,)

In [42]:
def build_model(input_shape=[0]):
    model = Sequential()
    model.add(Dense(1200,input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1000))
    model.add(Activation("softmax"))
    model.add(Dense(1))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam',
                                   metrics=['accuracy'])
    return model

In [43]:
model = KerasClassifier(build_fn=build_model, input_shape=x_train.shape[1::], verbose=1)

In [46]:
    epoch = 100
    batchsize = 8
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1)
    callbacks_list = [earlystop]
    history = model.fit(x_train, y_train, batch_size=batchsize, epochs=epoch,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks_list)
    # lossplot(history)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 1200)              127200    
_________________________________________________________________
activation_34 (Activation)   (None, 1200)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1200)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 1000)              1201000   
_________________________________________________________________
activation_35 (Activation)   (None, 1000)              0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 1001      
_________________________________________________________________
activation_36 (Activation)   (None, 1)               

KeyboardInterrupt: 

In [32]:
def lossplot(history):
    ori_val_Loss = history.history['val_loss']
    loss = history.history['loss']
    epochs = np.arange(len(history.epoch)) + 1
    plt.plot(epochs, ori_val_Loss, label='val loss')
    plt.plot(epochs, loss, label='loss')
    plt.title("Effect of model capacity on validation loss\n")
    plt.xlabel('Epoch #')
    plt.ylabel('Validation Loss')
    plt.legend()
    plt.show()

In [38]:
x_train, x_test, x_val, y_test, y_train, y_val = load_data()

(151, 105)
(1210, 105)
(151, 105)
(151, 1)
(1210, 1)
(151, 1)


In [45]:

def main():
    x_train, x_test, x_val, y_test, y_train, y_val = load_data()
    # print(x_train.shape[1::])
    model = KerasClassifier(build_fn=build_model, input_shape=x_train.shape[1::], verbose=0)
    epoch = 100
    batchsize = 8
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1)
    callbacks_list = [earlystop]
    history = model.fit(x_train, y_train, batch_size=batchsize, epochs=epoch,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks_list)